<a href="https://colab.research.google.com/github/siting1206/Plant_seedlings_classification/blob/main/Plant_seedlings_classification(ResNet50).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf
import pandas as pd

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

import pathlib
plant_seeds_dir = pathlib.Path('plant-seedlings-classification/train')
for dir in plant_seeds_dir.iterdir():
    print(str(dir))

seeds_list = list(plant_seeds_dir.glob('*/*.*'))
total_seed_count = len(list(plant_seeds_dir.glob('*/*.*')))
print('Total seed count : ', total_seed_count)

batch_size = 32
img_height = 224
img_width = 224

train_ds = keras.preprocessing.image_dataset_from_directory(
    plant_seeds_dir,
    batch_size = batch_size,
    seed=123,
    subset='training',
    validation_split=0.2,
    image_size=(img_height, img_width)
)

val_ds = keras.preprocessing.image_dataset_from_directory(
    plant_seeds_dir,
    batch_size = batch_size,
    seed=123,
    subset='validation',
    validation_split=0.2,
    image_size=(img_height, img_width)
)

class_names = train_ds.class_names
print(class_names)
print(len(class_names))

image_batch, label_batch = next(iter(train_ds))

print(image_batch.shape)
print(label_batch.shape)

AUTOTUNE = tf.data.experimental.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(AUTOTUNE)
val_ds = val_ds.cache().prefetch(AUTOTUNE)

preprocess_input = keras.applications.resnet_v2.preprocess_input

INPUT_SHAPE = (224,224,3)

base_model = keras.applications.ResNet50V2(input_shape=INPUT_SHAPE,
                                           include_top=False,
                                           weights='imagenet')

base_model.trainable = False
print(len(base_model.trainable_variables))
base_model.summary()

data_augmentation = Sequential([
                                layers.experimental.preprocessing.RandomFlip('horizontal'),
                                layers.experimental.preprocessing.RandomZoom(0.1)
])

image_batch, label_batch = next(iter(train_ds))
print(image_batch.shape)
preprocessed_batch = base_model(image_batch)
print(preprocessed_batch.shape)

global_average_layer = layers.GlobalAveragePooling2D()
feature_batch_average = global_average_layer(preprocessed_batch)
print(feature_batch_average.shape)

prediction_layer = layers.Dense(len(class_names))
prediction_batch = prediction_layer(feature_batch_average)
print(prediction_batch.shape)

inputs = keras.Input(shape=INPUT_SHAPE)
x = preprocess_input(inputs)
x = data_augmentation(x)
x = base_model(x, training=False)
x = global_average_layer(x)
x = tf.keras.layers.Dropout(0.2)(x)
outputs = prediction_layer(x)
model = keras.Model(inputs, outputs)

print("Trainable Variables : ", len(model.trainable_variables))

model.summary()

model.compile(optimizer='adam',
              loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

epochs = 30
history = model.fit(train_ds,
                    validation_data=val_ds,
                    epochs=epochs)

def plot_history(history):
  plt.figure(figsize=(10,10))

  accuracy= history.history['accuracy']
  val_accuracy=history.history['val_accuracy']
  plt.subplot(1,2,1)
  plt.plot(accuracy, label='accuracy')
  plt.plot(val_accuracy, label='val_accuracy')
  plt.legend(loc='lower right')
  plt.title("Training vs Validation Accuracy")
  plt.savefig('accuracy.png')

  loss = history.history['loss']
  val_loss = history.history['val_loss']
  plt.subplot(1,2,2)
  plt.plot(loss, label='loss')
  plt.plot(val_loss, label='val_loss')
  plt.legend(loc='upper right')
  plt.title('Training vs Vakidation Accuracy')
  plt.savefig('loss.png')

plot_history(history)

test_dir = pathlib.Path('plant-seedlings-classification/test')
test_name, pred_name = [], []
for dir in test_dir.iterdir():
  image_path = str(dir)
  test_image = keras.preprocessing.image.load_img(image_path,
                                                  target_size=(img_height, img_width))
  img_array = keras.preprocessing.image.img_to_array(test_image)
  img_array = tf.expand_dims(img_array, axis=0)
  predictions = model.predict(img_array)
  predictions = predictions[0]
  label = np.argmax(tf.nn.sigmoid(predictions))
  test_name.append(dir.name)
  pred_name.append(class_names[label])

print("Number of test images : ", len(test_name))

results = pd.DataFrame({"file":test_name,
                        "species":pred_name})
results.to_csv("submission.csv",index=False)